
# Best Methods — Stages 1–3 (Gloss → Fluent English → On‑Demand Refine → Translate)

This notebook implements the **Best** (highest‑quality, cloud‑powered) methods for the first three stages of your pipeline:

1. **Stage 1 — Gloss → Fluent English (one‑shot LLM)**  
   The LLM converts gloss tokens to natural, grammatical English in a single step (includes auto‑refine).

2. **Stage 2 — On‑Demand Refine (button action)**  
   A stronger LLM editor pass that returns a **structured JSON** showing what changed and why.

3. **Stage 3 — Translation (English → target language)**  
   A high‑quality LLM translation with tone/style hints and entity preservation.

> **Prereqs (before running):**
> - `pip install openai python-dotenv` (or set `OPENAI_API_KEY` in your environment).
> - Your CV stage should output **gloss tokens** (list of strings). You can plug them into the helper below.


In [1]:

# If needed, uncomment to install packages in your environment:
!pip install --quiet openai python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:

# ---- Imports & API key setup ----
import os
import json
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

# OpenAI SDK (Responses API)
from openai import OpenAI

# Optional: load .env if you keep your key there
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# Expect OPENAI_API_KEY to be set in environment
assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY (e.g., in your shell or a .env file)."

client = OpenAI()  # picks up key from env


In [2]:

# Small pretty-printer for JSON output (for Stage 2)
def pretty_json(obj: Any) -> str:
    return json.dumps(obj, indent=2, ensure_ascii=False)



## Stage 1 — Gloss → Fluent English (LLM; one shot)

**What it does**  
- Accepts a list of **gloss tokens** (e.g., `["YESTERDAY", "STORE", "I", "GO"]`).  
- Produces a **natural English sentence** with correct word order, articles, tense, punctuation, and capitalization.  
- Preserves **names/numbers/fingerspelled tokens** when present (tip: you can tag them upstream).

**Why this is “Best”**  
- A high‑quality LLM handles contextual reordering and inserts missing function words, which rule‑based methods often miss.

**Latency tip**  
- Keep **temperature low** (e.g., `0.2`) for deterministic, stable phrasing in real time.


In [3]:

def gloss_to_english_llm(
    gloss_tokens: List[str],
    context_hint: Optional[str] = None,
    temperature: float = 0.2,
    model: str = "gpt-4o-mini"
) -> str:
    """Convert a list of gloss tokens into a fluent English sentence using an LLM.

    Parameters
    ----------
    gloss_tokens : list of str
        Tokens from the CV recognizer (e.g., WSASL/ASL/ISL glosses).
    context_hint : str, optional
        Optional short context (topic/domain) to help resolve ambiguous glosses.
    temperature : float
        Lower values → more deterministic output.
    model : str
        OpenAI model to use.

    Returns
    -------
    str
        A single, natural English sentence ending with proper punctuation.
    """
    # 1) Prepare the instruction; we specify constraints to preserve semantics.
    system = (
        "You are a precise sign-language translator. "
        "Convert gloss tokens to a natural English sentence with correct grammar, "
        "articles, tense, and punctuation. Do not add new facts beyond what is implied "
        "by the glosses. Preserve named entities and numbers exactly. Output only the sentence."
    )

    # 2) Provide a couple of tiny few-shot examples to stabilize style.
    examples = [
        {
            "gloss": ["YESTERDAY", "STORE", "I", "GO"],
            "english": "I went to the store yesterday."
        },
        {
            "gloss": ["TODAY", "SCHOOL", "WE", "MEET", "AFTERNOON"],
            "english": "We will meet at school this afternoon."
        }
    ]

    # 3) Compose user content with the current tokens and optional context.
    user_lines = []
    if context_hint:
        user_lines.append(f"Context: {context_hint}")
    user_lines.append("Gloss tokens: " + " ".join(gloss_tokens))
    user_lines.append("Return a single complete English sentence.")
    user_prompt = "\n".join(user_lines)

    # 4) Build an input string combining instruction + few-shots + current request.
    #    Using the Responses API with `input` keeps things simple.
    few_shot_text = "\n\n".join([
        f"Example gloss: {' '.join(ex['gloss'])}\nExample English: {ex['english']}"
        for ex in examples
    ])

    full_input = (
        f"System: {system}\n\n"
        f"{few_shot_text}\n\n"
        f"Now, translate the following into one fluent English sentence.\n"
        f"{user_prompt}"
    )

    # 5) Call the model. Keep temperature low for consistency.
    resp = client.responses.create(
        model=model,
        input=full_input,
        temperature=temperature
    )

    # 6) Extract the plain text. The SDK exposes output_text for convenience.
    sentence = resp.output_text.strip()

    # 7) Ensure final punctuation.
    if not sentence.endswith(('.', '!', '?')):
        sentence += "."
    return sentence


In [4]:

# --- Demo (replace with your CV output) ---
demo_gloss = ["YESTERDAY", "STORE", "I", "GO"]
english_sentence = gloss_to_english_llm(demo_gloss, context_hint="Daily activities")
print(english_sentence)


I went to the store yesterday.



## Stage 2 — On‑Demand Refine (LLM editor with JSON diff)

**What it does**  
- Given a sentence (typically the Stage‑1 output), returns a **refined** version and a **list of changes**.  
- Uses **structured JSON** so your UI can show a “what changed” diff.

**Why this is “Best”**  
- Provides higher fluency and user‑trust via transparent edits.  
- We ask the model to **return JSON only** and use `response_format={"type": "json_object"}` to ensure valid JSON.

**Schema we expect**  
```json
{
  "clean_text": "string",
  "changes": [
    {"type": "grammar|style|punctuation", "before": "str", "after": "str", "explanation": "str"}
  ]
}
```


In [7]:

def refine_sentence_with_changes(
    sentence: str,
    temperature: float = 0.2,
    model: str = "gpt-4o-mini"
) -> Dict[str, Any]:
    """Refine a sentence and return a structured JSON with the clean text and changes.

    Returns
    -------
    dict with keys:
      - clean_text: str
      - changes: list[dict] with keys {type, before, after, explanation}
    """
    system = (
        "You are a careful, minimal editor. Improve grammar, clarity, and punctuation "
        "without changing the meaning. Preserve names and numbers exactly. "
        "Return only JSON; do not include any extra commentary."
    )

    # We instruct the model to follow our JSON shape and enforce JSON output.
    prompt = (
        "Edit the following sentence. Return JSON with keys: "
        "'clean_text' (string) and 'changes' (array of objects with keys "
        "'type', 'before', 'after', 'explanation').\n\n"
        f"Sentence: {sentence}"
    )

    resp = client.responses.create(
        model=model,
        input=f"System: {system}\n\nUser: {prompt}",
        temperature=temperature,
        #response_format={ "type": "json_object" }  # ensures valid JSON
    )

    # Parse the JSON from the text output.
    data = json.loads(resp.output_text)

    # Minimal sanity checks
    data.setdefault("clean_text", sentence)
    data.setdefault("changes", [])
    return data


In [8]:

# --- Demo for Stage 2 ---
to_refine = english_sentence  # from Stage 1
refined = refine_sentence_with_changes(to_refine)
print(pretty_json(refined))


{
  "clean_text": "I went to the store yesterday.",
  "changes": []
}



## Stage 3 — Translation (English → target language)

**What it does**  
- Translates the (optionally refined) English sentence into a **target language** chosen by the user.  
- Preserves named entities and numbers; supports tone (`formal` / `informal`) and optional locale hints.

**Why this is “Best”**  
- High‑quality LLM translation is robust to varied syntax and domain contexts.

**Usage**  
- Call `translate_text_llm(text, target_lang="hi", tone="formal")`  
- `target_lang` can be a **language name** (`"Hindi"`) or a **BCP‑47/ISO code** (`"hi"`).


In [9]:

def translate_text_llm(
    text: str,
    target_lang: str,
    tone: str = "neutral",
    locale_hint: Optional[str] = None,
    temperature: float = 0.2,
    model: str = "gpt-4o-mini"
) -> str:
    """Translate English text to a target language using an LLM.

    Parameters
    ----------
    text : str
        The English source sentence.
    target_lang : str
        Language name or code (e.g., 'Hindi' or 'hi').
    tone : str
        'formal', 'informal', or 'neutral' (hint only).
    locale_hint : str, optional
        Optional locale/region style (e.g., 'IN' for India).
    temperature : float
        Keep low for consistent, literal translations when desired.
    model : str
        OpenAI model to use.

    Returns
    -------
    str
        The translated sentence only (no extra commentary).
    """
    system = (
        "You are a professional translator. Translate the user's text to the requested "
        "target language. Preserve names and numbers exactly. Do not add explanations. "
        "Return only the translated sentence."
    )

    hints = [
        f"Target language: {target_lang}",
        f"Tone: {tone}"
    ]
    if locale_hint:
        hints.append(f"Locale hint: {locale_hint}")
    hint_text = " | ".join(hints)

    prompt = f"{hint_text}\n\nText:\n{text}"

    resp = client.responses.create(
        model=model,
        input=f"System: {system}\n\nUser: {prompt}",
        temperature=temperature
    )
    return resp.output_text.strip()


In [10]:

# --- Demo for Stage 3 ---
final_english = refined.get("clean_text", english_sentence)
translated_hi = translate_text_llm(final_english, target_lang="Hindi", tone="formal", locale_hint="IN")
print("EN  :", final_english)
print("HI  :", translated_hi)


EN  : I went to the store yesterday.
HI  : मैं कल दुकान गया था।



### (Optional) Convenience wrapper — run Stages 1→2→3

Use `run_best_pipeline` when you want to go end‑to‑end for a single sentence.  
For **real‑time**, you should call Stage 1 for each finalized chunk, and then trigger Stage 2 only when the user taps **Refine**.


In [11]:

@dataclass
class BestPipelineResult:
    english: str
    refined: Dict[str, Any]
    translated: str

def run_best_pipeline(
    gloss_tokens: List[str],
    target_lang: str,
    tone: str = "neutral",
    locale_hint: Optional[str] = None,
    do_refine: bool = True
) -> BestPipelineResult:
    """Run Stages 1→2→3 for a single sentence/chunk."""
    # Stage 1
    english = gloss_to_english_llm(gloss_tokens)

    # Stage 2 (optional button in the UI)
    refined = refine_sentence_with_changes(english) if do_refine else {"clean_text": english, "changes": []}

    # Stage 3
    translated = translate_text_llm(refined["clean_text"], target_lang=target_lang, tone=tone, locale_hint=locale_hint)

    return BestPipelineResult(english=english, refined=refined, translated=translated)


In [ ]:

result = run_best_pipeline(["TODAY", "SCHOOL", "WE", "MEET", "AFTERNOON"], target_lang="Hindi", tone="formal", locale_hint="IN")
print("Stage 1 (English):", result.english)
print("\nStage 2 (Refined JSON):\n", pretty_json(result.refined))
print("\nStage 3 (Translated):", result.translated)


Stage 1 (English): We will meet at school this afternoon.

Stage 2 (Refined JSON):
 {
  "clean_text": "We will meet at school this afternoon.",
  "changes": []
}

Stage 3 (Translated): हम इस दोपहर स्कूल में मिलेंगे।



### Notes & Tips

- **Boundaries:** Call Stage 1 only on **finalized chunks** (e.g., clause/sentence boundaries) from your CV pipeline for stable output.
- **Caching:** Add a dict cache keyed by the sentence string to avoid repeated LLM calls during demos.
- **Tone Controls:** For translation, you can set `tone="formal"` or `"informal"` based on user preference.
- **Entity Protection:** Upstream, you can wrap entities like `«Vedant»` so the model keeps casing; then strip markers after translation.
- **Costs & Latency:** Keep `temperature=0.2` and sentences short; batch only when safe. Consider streaming for UI responsiveness.
